In [1]:
import sys
import os

# Add the root directory of your project to the Python path
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import einops
import torch
from torch.optim.lr_scheduler import LambdaLR
from models import *
from branch_datasets import *
from torch.utils.data import DataLoader, Dataset

from torch import Tensor
from typing import Optional, Tuple
from jaxtyping import Float, Int

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.offline as pyo


In [3]:
# Define Config

In [4]:
length = 50 # number of tokens to remember
vocab_size=128
d_model=192
d_ffn=768
h=1
n=2
max_len = 3*length + 1
num_epochs = 4000
model = Transformer(vocab_size, d_model, d_ffn, h, n, max_len, attn_only=True, layer_norm=False).to("cuda")

In [5]:
# Dataset

In [6]:

train_dataset = InductionHeadsDataset(4096, length, vocab_size)
test_dataset = InductionHeadsDataset(1024, length, vocab_size)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [7]:
# Train

In [8]:
cross_entropy = torch.nn.CrossEntropyLoss()

def loss_fn(logits: Float[Tensor, "batch 3*length+1 d_model"], tokens: Float[Tensor, "batch 3*length+1"], gaps: Int[Tensor, "batch"]) -> Float:
    # Compute the start indices for slicing in a vectorized way
    start_indices = gaps + length + 1
    index_array = torch.arange(length-1).to(logits.device) + start_indices[:, None]
    # Use advanced indexing to gather the appropriate slices
    masked_logits = logits[torch.arange(logits.shape[0])[:, None, None], index_array[:, :, None], torch.arange(logits.shape[2])[None, None, :]]
    masked_logits = masked_logits.to(logits.device)
    tokens = tokens[:, 2:length+1]
    tokens = tokens.reshape(-1)
    masked_logits = masked_logits.reshape(-1, logits.size(-1))
    loss = cross_entropy(masked_logits, tokens)
    return loss

In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=0.01) # Change weight decay if necessary
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda i: min(i/100, 1))
losses = []

for epoch in range(num_epochs):
    tokens, gaps = next(iter(train_dataloader))
    tokens, gaps = tokens.cuda(), gaps.cuda()
    logits = model(tokens)
    loss = loss_fn(logits, tokens, gaps)
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    scheduler.step()
    if epoch % 10 == 0 or epoch < 20:
        print(f'Epoch {epoch}: {loss.item()}')
    

Epoch 0: 5.254245281219482
Epoch 1: 5.276902198791504
Epoch 2: 5.24575662612915
Epoch 3: 5.276229381561279
Epoch 4: 5.258651256561279
Epoch 5: 5.2356696128845215
Epoch 6: 5.199569225311279
Epoch 7: 5.188490390777588
Epoch 8: 5.147525787353516
Epoch 9: 5.136970043182373
Epoch 10: 5.145175457000732
Epoch 11: 5.10391092300415
Epoch 12: 5.073296546936035
Epoch 13: 5.0774664878845215
Epoch 14: 5.056448936462402
Epoch 15: 5.017152309417725
Epoch 16: 5.039661407470703
Epoch 17: 5.0255126953125
Epoch 18: 5.0294976234436035
Epoch 19: 5.0131683349609375
Epoch 20: 5.011415481567383
Epoch 30: 4.968501091003418
Epoch 40: 4.954255104064941
Epoch 50: 4.951467037200928
Epoch 60: 4.908801078796387
Epoch 70: 4.916306018829346
Epoch 80: 4.899270057678223
Epoch 90: 4.895861625671387
Epoch 100: 4.892274856567383
Epoch 110: 4.886834621429443
Epoch 120: 4.891945838928223
Epoch 130: 4.8754448890686035
Epoch 140: 4.871891021728516
Epoch 150: 4.878665447235107
Epoch 160: 4.86287260055542
Epoch 170: 4.8770337104

In [10]:
# Plot the loss curve with plotly library

pyo.init_notebook_mode(connected=True)


fig = go.Figure()
fig.add_trace(go.Scatter(y=losses))
fig.add_trace(go.Scatter(
    x=list(range(len(losses))),
    y=losses,
    mode='lines+markers',
    name='Loss',
    hoverinfo='x+y',
    line=dict(color='blue', width=2)
))

# Add titles and labels
fig.update_layout(
    title='Training Loss Over Epochs',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    hovermode='x unified'
)

# Show the plot
pyo.iplot(fig)

In [11]:
# Test

In [12]:
# generate some tokens from the test dataloader
model.eval()
tokens, gaps = next(iter(test_dataloader))
tokens = tokens.cuda()[:1]
logits = model(tokens)
generated_tokens = torch.argmax(logits, dim=-1)
print(tokens)
print(f'Generated tokens: {generated_tokens}')

tensor([[  0,  86,  47,  75, 105,  82,  30,  71,  79,  73,  19, 126, 126, 103,
          96,  59,  31, 126,  55,  42,  37,   5,  57,  24,  43,  10,  38,  19,
          45, 115,  74,  96,  26,   7,  27,   0, 113, 111,  40,  38,  38,  10,
          79, 119,  81,  19,  61, 120,  93, 111, 111,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,  86,  47,  75, 105,  82,  30,
          71,  79,  73,  19, 126, 126, 103,  96,  59,  31, 126,  55,  42,  37,
           5,  57,  24,  43,  10,  38,  19,  45, 115,  74,  96,  26,   7,  27,
           0, 113, 111,  40,  38,  38,  10,  79, 119,  81,  19,  61, 120,  93,
         111, 111,   0,   0,   0,   0,   0,   0,   0,   0,   0]],
       device='cuda:0')
Generated tokens: tensor([[  0,  86,  86,  47,  47,  47,  86,  47,  47,  47,  86,  47,  47,  47,
          47,  47,  86,

In [13]:
# Visualize Attention

In [14]:
print(model)

Transformer(
  (W_E): Embedding(128, 192)
  (layers): ModuleList(
    (0-1): 2 x DecoderLayer(
      (attn): SubLayer(
        (layer_fn): MultiHeadAttention(
          (linears): Linear(in_features=192, out_features=576, bias=True)
          (O): Linear(in_features=192, out_features=192, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
  (pos): PositionalEncoding()
  (W_U): Linear(in_features=192, out_features=128, bias=True)
)


In [15]:
for head in range(h):
    # view OV circuit
    ov_circuit = model.cache["W_E"] @ model.layers[0].attn.layer_fn.cache['W_OV'][head].T @ model.cache["W_U"].T # (vocab_size, vocab_size)
    plotly_fig = go.Figure(data=go.Heatmap(z=ov_circuit.numpy()))
    plotly_fig.update_layout(title=f'OV Circuit {head}')
    pyo.iplot(plotly_fig)
    

In [16]:
def mask_scores(attn_scores: Float[Tensor, "heads length length"]) -> Float[Tensor, "heads length length"]:
    '''Mask the attention scores so that tokens don't attend to previous tokens.'''
    mask = torch.tril(torch.ones_like(attn_scores)).bool()
    neg_inf = torch.tensor(-1.0e6).to(attn_scores.device)
    masked_attn_scores = torch.where(mask, attn_scores, neg_inf)
    return masked_attn_scores


for head in range(h):
    # view OV circuit
    W_pos = model.cache["W_pos"]
    pos_by_pos_scores = W_pos @ model.layers[0].attn.layer_fn.cache['W_QK'][head] @ W_pos.T # (vocab_size, vocab_size)
    masked_scaled = mask_scores(pos_by_pos_scores / ((d_model // h) ** 0.5))
    pos_by_pos_pattern = torch.softmax(masked_scaled, dim=-1)
    plotly_fig = go.Figure(data=go.Heatmap(z=pos_by_pos_pattern.numpy()))
    plotly_fig.update_layout(title=f'QK Circuit {head}')
    pyo.iplot(plotly_fig)

In [17]:
for head in range(h):
    # view layer 0
    head_attn = model.layers[0].attn.layer_fn.cache['attn'][0, head] # (vocab_size, vocab_size)
    plotly_fig = go.Figure(data=go.Heatmap(z=head_attn.numpy()))
    plotly_fig.update_layout(title=f'Attention head {head}')
    pyo.iplot(plotly_fig)

for head in range(h):
    # view layer 1
    head_attn = model.layers[1].attn.layer_fn.cache['attn'][0, head] # (vocab_size, vocab_size)
    plotly_fig = go.Figure(data=go.Heatmap(z=head_attn.numpy()))
    plotly_fig.update_layout(title=f'Attention head {head}')
    pyo.iplot(plotly_fig)